In [1]:
import sys
import ee
%run Sentinel2b.ipynb
ee.Authenticate()
ee.Initialize(project='plottosat-fjgibson')

# Define study region
countries = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')


modulename = 'ee'
if modulename not in sys.modules:
    # import GEE and Authenticate, token or log in will be asked from web browser
    import ee
    # TO DO : UNCOMMENT THIS IF IT IS THE FIRST TIME AUTHINTICATE GEE ON THIS MACHINE
    # ee.Authenticate()
    ee.Initialize()
# else:
    # google earth engine already imported and authenticated


modulename = 'ipynb_MapVisualisation'
if modulename not in sys.modules:
    %run MapVisualisation.ipynb
    # adding an identifier to sys.modules to avoiding loading the same file multiple times
    sys.modules['ipynb_MapVisualisation'] = None
# else
    # Utils modules has already been loaded somewhere else


modulename = 'ipynb_Sentinel2b'
if modulename not in sys.modules:
    %run Sentinel2b.ipynb
    # adding an identifier to sys.modules to avoiding loading the same file multiple times
    sys.modules['ipynb_Sentinel2b'] = None
# else
    # Utils modules has already been loaded somewhere else


Utils imported
Masks class imported
Sentinel2b class imported
MapVisualisation class imported - uses folium library
Sentinel2b class imported


In [2]:
from indices import INDEX_NAMES
import indices

In [3]:

# Area of Interest
# AOI = ee.Geometry.Polygon(
#         [[[-2.97404756802236, 41.99472813320794],
#           [-2.97404756802236, 41.8107480842418],
#           [-2.66643038052236, 41.8107480842418],
#           [-2.66643038052236, 41.99472813320794]]])
long = 0.0
lat = 51.5
AOI = ee.Geometry.Polygon(
        [[[long-5.1, lat+5.1],
          [long-5.1, lat-5.1],
          [long+5.1, lat-5.1],
          [long+5.1, lat+5.1]]])

# cloudmask01 = getCloudMask(median)

START_DATE = '2019-01-01'
END_DATE = '2019-12-31'



masks = { 'gsw': 1}#,'forestMask': {'buffer':10, 'startDate':'2000-01-01', 'endDate':'2022-12-31',}} # # , 'lmask': 30,

printAvailableIndicesS2()

sentinel2Info = {
    "identifier": 'sentinel-2',                                              
    "collection": "COPERNICUS/S2_SR_HARMONIZED",
    #"launchDate" : "2015-05-23",
    #"retirmentDate":todayDate, #not retired yet
    "startDate"   : '2019-01-01',
    "endDate"     : '2019-12-31',
    #"added"     : False,
    "clouds"    : 50,
    "selectedBand": ['B4', 'B3', 'B2', 'B8'],
    "selectedIndices": ["ndvi"] 
} 


s2 = Sentinel2(AOI, sentinel2Info, masks)


s2.byMonth(2020)

median = s2.getMedian()
bandNames = median.bandNames()
print('Band names medianS2: ', bandNames.getInfo())

# Code used for testing cloud masks - Cloud masking used by default so no need to worry about this code
med = s2.getCollection().first()
clouds = med.select('clouds').selfMask()
shadows = med.select('shadows').selfMask()
dark_pixels = med.select('dark_pixels')  # .selfMask()
probability = med.select('probability')
cloudmask = med.select('cloudmask')
cloudmaskMasked = cloudmask  # .selfMask()
cloud_transform = med.select('cloud_transform')

# Selecting RGB bands for visualisation purposes
medianS2 = median.select(['B4', 'B3', 'B2', 'B8'])


# Calculate NDVI on the median image
image = median.select(['B4', 'B8'])


# for index in INDEX_NAMES:
for index in ["ndvi"]:
    f_add_index = getattr(indices, f"add_{index}")
    image = f_add_index(image)
    
    
# export image  to GEE - adding maxPixels is a bad habit, used carefully
task = ee.batch.Export.image.toDrive(**{
    'image': image,
    'description': 'medRGB1_GB1_2019',
    'folder': 'earth_engine_demos_all_indices',
    'scale': 10,
    'region': AOI.getInfo()['coordinates'],
    'maxPixels': 1549491660
})
task.start()


    Key                                   Index                Formula
0  ndvi  Normalised Difference Vegetation Index  (B8 - B4) / (B8 + B4)
selectedBandsNIndices: ['B4', 'B3', 'B2', 'B8', 'ndvi']


/home/finley/Work/RDS/projects/greensight/.venv/lib/python3.12/site-packages/ee/deprecation.py:207: DeprecationWarning: 

Attention required for UMD/hansen/global_forest_change_2022_v1_10! You are using a deprecated asset.
To ensure continued functionality, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/UMD_hansen_global_forest_change_2022_v1_10

  warnings.warn(warning, category=DeprecationWarning)


Band names medianS2:  []


In [4]:
%run MapVisualisation.ipynb
# These are the coordinates of Spain
my_map = folium.Map(location=[lat, long], zoom_start=11)
# Add custom basemaps
basemaps['Google Maps'].add_to(my_map)
basemaps['Google Satellite Hybrid'].add_to(my_map)


def update_cld_shdw_mask(img):
    print('test')



# Add Land Mask to the map
my_map.add_ee_layer(image, {
                    'bands': ["ndvi"], 'min': 0, 'max': 2500, 'gamma': 1.1}, 'median')
my_map.add_ee_layer(AOI, {}, 'AOI')



# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())
plugins.Fullscreen().add_to(my_map)

# Display the map.
display(my_map)


MapVisualisation class imported - uses folium library
Could not display median


In [21]:
image.bandNames().getInfo()

EEException: Image.select: Band pattern 'B4' was applied to an Image with no bands. See https://developers.google.com/earth-engine/guides/debugging#no-bands